In [1]:
!pip install torch torchvision
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
# import torchvision.transforms as transforms
from torchvision import datasets, transforms
import torch.optim as optim

---

# 📘 Albumentations Augmentation Guide

Albumentations is a **fast and flexible image augmentation library** for computer vision. It is widely used with **PyTorch, TensorFlow, and Keras** for improving generalization of models.

---

## 🔹 Why use augmentations?

* Prevent **overfitting** by exposing the model to varied data.
* Improve **robustness** to transformations like flips, rotations, noise, or occlusions.
* Simulate real-world conditions (e.g., missing pixels, lighting changes, distortions).

---

## 🔹 Commonly Used Transforms

### 1. **Horizontal Flip**

```python
A.HorizontalFlip(p=0.5)
```

* Randomly flips the image left-to-right.
* Helps in tasks where orientation does not matter (e.g., classification, segmentation).

---

### 2. **ShiftScaleRotate**

```python
A.ShiftScaleRotate(
    shift_limit=0.0625,  # Max fraction for shifting image (e.g., 6.25%)
    scale_limit=0.1,     # Zoom in/out up to ±10%
    rotate_limit=45,     # Rotate within ±45 degrees
    p=0.5
)
```

* Applies random **translation (shift)**, **zoom (scale)**, and **rotation**.
* Useful in making models invariant to small positional changes.

---

### 3. **CoarseDropout**

```python
A.CoarseDropout(
    max_holes=1,       # max number of dropped regions
    max_height=16,     # max height of dropped block
    max_width=16,      # max width of dropped block
    min_holes=1,       # min number of dropped regions
    min_height=16,     # min height of dropped block
    min_width=16,      # min width of dropped block
    fill_value=(123.68, 116.78, 103.94),  # dataset mean
    mask_fill_value=None,
    p=0.5
)
```

* Randomly "drops out" square/rectangular regions from the image.
* Similar to **Cutout regularization**.
* Forces the model to rely on surrounding context instead of only specific pixels.

---

## 🔹 Example Pipeline

```python
import albumentations as A

mean = (123.68, 116.78, 103.94)  # dataset mean (replace with yours)

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.5),
    A.CoarseDropout(
        max_holes=1, max_height=16, max_width=16,
        min_holes=1, min_height=16, min_width=16,
        fill_value=mean, mask_fill_value=None, p=0.5
    )
])
```

---

## 🔹 Tips for Use

* Always calculate your **dataset mean and std** before setting `fill_value`.
* Use different `p` values to control augmentation probability.
* When working with segmentation, remember to also pass `mask` alongside `image` in `transform(image=image, mask=mask)`.
* Apply augmentations **only on training data**, not validation/test.

---

## 🔹 Resources

* [Albumentations Docs](https://albumentations.ai/docs/)
* [Albumentations GitHub](https://github.com/albumentations-team/albumentations)
* Research paper: *Albumentations: Fast and Flexible Image Augmentations* (Buslaev et al., 2020)

---



In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
dropout_value = 0.0001
class ciferNet(nn.Module):
    def __init__(self):
        super(ciferNet, self).__init__()

        # Conv Block 1
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(dropout_value),
        )

        # Conv Block 2
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(dropout_value),
        )

        # Conv Block 3
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Dropout(dropout_value),
        )

        # Conv Block 4 → Depthwise separable convolution
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1, groups=128, bias=False),  # Depthwise conv
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(1, 1), bias=False),  # Pointwise conv
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Dropout(dropout_value),
        )

        # GAP and classifier
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Conv2d(in_channels=256, out_channels=10, kernel_size=(1, 1), bias=False)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.gap(x)
        x = self.classifier(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x, dim=-1)


In [11]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = ciferNet().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
            Conv2d-4           [-1, 32, 16, 16]           4,608
              ReLU-5           [-1, 32, 16, 16]               0
       BatchNorm2d-6           [-1, 32, 16, 16]              64
           Dropout-7           [-1, 32, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          18,432
              ReLU-9           [-1, 64, 16, 16]               0
      BatchNorm2d-10           [-1, 64, 16, 16]             128
           Conv2d-11             [-1, 64, 8, 8]          36,864
             ReLU-12             [-1, 64, 8, 8]               0
      BatchNorm2d-13             [-1, 64, 8, 8]             128
          Dropout-14             [

In [12]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Train Phase
train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.0625,
        scale_limit=0.1,
        rotate_limit=45,
        p=0.5
    ),
    A.CoarseDropout(
        max_holes=1,
        max_height=16,
        max_width=16,
        min_holes=1,
        min_height=16,
        min_width=16,
        fill_value=(123.68, 116.78, 103.94),
        mask_fill_value=None,
        p=0.5
    ),
    A.Normalize(mean=(0.4741, 0.4727, 0.4733),
                std=(0.2521, 0.2520, 0.2506)),
    ToTensorV2()
])

# Test Phase
test_transforms = A.Compose([
    A.Normalize(mean=(0.4741, 0.4727, 0.4733),
                std=(0.2521, 0.2520, 0.2506)),
    ToTensorV2()
])


/tmp/ipython-input-306781532.py:13: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(


In [13]:
from torch.utils.data import Dataset

class CIFAR10Albumentations(Dataset):
    def __init__(self, root, train=True, download=False, transform=None):
        self.cifar10 = datasets.CIFAR10(root=root, train=train, download=download)
        self.transform = transform

    def __getitem__(self, index):
        image, label = self.cifar10[index]
        if self.transform:
            # Albumentations requires the image to be passed as a keyword argument 'image'
            image = self.transform(image=np.array(image))['image'] # Convert PIL to numpy array before applying transform
        return image, label

    def __len__(self):
        return len(self.cifar10)

In [14]:
import numpy as np

train = CIFAR10Albumentations('./data', train=True, download=True, transform=train_transforms)
test = CIFAR10Albumentations('./data', train=False, download=True, transform=test_transforms)

In [15]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
train_dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
test_dataloader_args = dict(shuffle=False, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=False, batch_size=64)


# train dataloader
train_loader = torch.utils.data.DataLoader(train, **train_dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **test_dataloader_args)

CUDA Available? True


In [16]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss.item()) # Added .item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.float().argmax(dim=1, keepdim=True)  # get the index of the max log-probability, convert to float
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss) # Added .item()

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [19]:
from torch.optim.lr_scheduler import StepLR

model =  ciferNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# scheduler = StepLR(optimizer, step_size=6, gamma=0.1)


EPOCHS = 35
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    # scheduler.step()
    test(model, device, test_loader)

EPOCH: 0


Loss=1.531897783279419 Batch_id=390 Accuracy=40.73: 100%|██████████| 391/391 [00:21<00:00, 17.86it/s]



Test set: Average loss: 1.4204, Accuracy: 5055/10000 (50.55%)

EPOCH: 1


Loss=1.1379144191741943 Batch_id=390 Accuracy=53.45: 100%|██████████| 391/391 [00:22<00:00, 17.58it/s]



Test set: Average loss: 1.1288, Accuracy: 5845/10000 (58.45%)

EPOCH: 2


...


Test set: Average loss: 0.4671, Accuracy: 8444/10000 (84.44%)

EPOCH: 33


Loss=0.48013001680374146 Batch_id=390 Accuracy=83.73: 100%|██████████| 391/391 [00:20<00:00, 18.74it/s]



Test set: Average loss: 0.4654, Accuracy: 8448/10000 (84.48%)

EPOCH: 34


Loss=0.4376187324523926 Batch_id=390 Accuracy=83.77: 100%|██████████| 391/391 [00:21<00:00, 18.26it/s]



Test set: Average loss: 0.4711, Accuracy: 8382/10000 (83.82%)

